## 建立你的深度神经网络：step by step
   微专业第一门课神经网络第四章的第二个作业，构建你自己的神经网络。
   在这之前，你已经训练过两层神经网络（即只有单隐藏层的神经网络），这次作业，你将建立一个深层神经网络。
   在这节中，你将实现所有建立深度神经网络必须实现的方法，并且把这些方法运用到下一个作业week4_03中，用于图像识别。
    
在这次编程练习之后，你将学会：
1. 使用非线性单元比如ReLU来实现你的模型
2. 建立一个更深一点的神经网络（大于一个隐藏层）
3. 实现一个简单好用的神经网络class
注释：
上标[l]表示这是关于神经网络第l层的数
上标（i）表示这是第几个训练样本
下脚标i表示这个某一层的第i个隐藏单元

## 1 packages
首先引进这些包，numpy，matplotlib已经说过了
dnn_utils 提供了一些必要的方法后面会用到的
testCases提供了一些测试用例来检测你的方法是否实现正确
np.random.seed(1)是用来保证所有的随机函数调用的一致性

In [3]:
import numpy as np
import h5py
import matplotlib.pyplot as plt
from testCases_v2 import *
from dnn_utils_v2 import sigmoid, sigmoid_backward, relu, relu_backward

%matplotlib inline
plt.rcParams['figure.figsize'] = (5.0, 4.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2

np.random.seed(1)

## 2 任务大纲
为了建立你的神经网络，你将实现几个“helper functions”，这些辅助函数将会用到下一个任务中去建立一个两层神经网络和一个L层神经网络。
outline of assignment：
1. 初始化一个双层神经网络的参数以及一个L层神经网络的参数
2. 实现正向传播模块：
    （1）完成正向传播的线性部分（得到Z[l]结果）
    （2）使用激活函数（ReLU、sigmoid）
    （3）将前面两步结合起来（Linear->Activation）正向函数
    （4）将前面L-1层（Linear->ReLU）重复L-1次，再加上（Linear->Sigmoid）在最后一层，这些会得到一个新的L_model_forward函数
3. 计算损失
4. 实现反向传播过程
     （1）计算一层的反向传播Linear部分
     （2）加载的工具包里面会提供关于ReLU以及sigmoid的导数计算方法
     （3）结合前面两步（Linear->Activation）
     （4）前面L-1层使用的ReLU函数，最后一层为sigmoid，建立一个新的L_model_backward函数
5. 更新参数

注意：对于每一个正向传播的过程，都会有一个对应的反向传播过程，这也是为什么在每一步正向传播模块中需要把一些值存储在cache中。这些cache中的值在反向传播过程中计算梯度非常重要。

## 3 初始化
接下来要写两个辅助函数来初始化参数，第一个函数用来初始化双层神经网络的参数，另一个用来生成L层神经网络的初始化过程。
### 3.1 双层神经网络
练习：创建并且初始化双层神经网络的参数
提示：使用random初始化权重矩阵。使用np.random.randn(shape)\*0.01即W的初始化，关于参数b，使用0来初始化 np.zeros(shape).

In [4]:
# GRADED FUNCTION: initialize_parameters

def initialize_parameters(n_x, n_h, n_y):
    """
    Argument:
    n_x -- size of the input layer
    n_h -- size of the hidden layer
    n_y -- size of the output layer
    
    Returns:
    parameters -- python dictionary containing your parameters:
                    W1 -- weight matrix of shape (n_h, n_x)
                    b1 -- bias vector of shape (n_h, 1)
                    W2 -- weight matrix of shape (n_y, n_h)
                    b2 -- bias vector of shape (n_y, 1)
    """
    
    np.random.seed(1)
    
    ### START CODE HERE ### (≈ 4 lines of code)
    W1 = np.random.randn(n_h,n_x)*0.01
    b1 = np.zeros((n_h,1))
    W2 = np.random.randn(n_y,n_h)*0.01
    b2 = np.zeros((n_y,1))
    ### END CODE HERE ###
    
    assert(W1.shape == (n_h, n_x))
    assert(b1.shape == (n_h, 1))
    assert(W2.shape == (n_y, n_h))
    assert(b2.shape == (n_y, 1))
    
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameters    

In [5]:
parameters = initialize_parameters(2,2,1)
print("W1 = " + str(parameters["W1"]))
print("b1 = " + str(parameters["b1"]))
print("W2 = " + str(parameters["W2"]))
print("b2 = " + str(parameters["b2"]))

W1 = [[ 0.01624345 -0.00611756]
 [-0.00528172 -0.01072969]]
b1 = [[ 0.]
 [ 0.]]
W2 = [[ 0.00865408 -0.02301539]]
b2 = [[ 0.]]


### 3.2 L层神经网络
对深度L层神经网络的初始化要比双层神经网络参数初始化复杂的多，因为有更多的权重矩阵以及b。当你实现 initialize_parameters_deep，你需要确保每一层的矩阵的维度。
![深层神经网络每层参数维度例子](http://upload-images.jianshu.io/upload_images/5355764-afc79376df8cbaac.png?imageMogr2/auto-orient/strip%7CimageView2/2/w/1240)
提示：
1. 这里深度神经网络前L-1层的激活函数使用的ReLU函数，最后一层使用的是sigmoid
2. 我们需要把每一层的隐藏单元数量存在一个变量layer_dims里面，根据这个列表，我们更方便按照规则初始化参数w和b。
3. 下面是L=1(单层神经网络)的实现，帮助你去实现一般情况。
```python
if L == 1:
      parameters["W" + str(L)] = np.random.randn(layer_dims[1], layer_dims[0]) * 0.01
      parameters["b" + str(L)] = np.zeros((layer_dims[1], 1))
```

In [6]:
# GRADED FUNCTION: initialize_parameters_deep

def initialize_parameters_deep(layer_dims):
    """
    Arguments:
    layer_dims -- python array (list) containing the dimensions of each layer in our network
    
    Returns:
    parameters -- python dictionary containing your parameters "W1", "b1", ..., "WL", "bL":
                    Wl -- weight matrix of shape (layer_dims[l], layer_dims[l-1])
                    bl -- bias vector of shape (layer_dims[l], 1)
    """
    
    np.random.seed(3)
    parameters = {}
    L = len(layer_dims)            # number of layers in the network

    for l in range(1, L):
        ### START CODE HERE ### (≈ 2 lines of code)
        parameters['W' + str(l)] = np.random.randn(layer_dims[l], layer_dims[l-1]) * 0.01

        parameters['b' + str(l)] = np.zeros((layer_dims[l], 1))

        ### END CODE HERE ###
        
        assert(parameters['W' + str(l)].shape == (layer_dims[l], layer_dims[l-1]))
        assert(parameters['b' + str(l)].shape == (layer_dims[l], 1))
        
    return parameters

In [7]:
parameters = initialize_parameters_deep([5,4,3])
print("W1 = " + str(parameters["W1"]))
print("b1 = " + str(parameters["b1"]))
print("W2 = " + str(parameters["W2"]))
print("b2 = " + str(parameters["b2"]))

W1 = [[ 0.01788628  0.0043651   0.00096497 -0.01863493 -0.00277388]
 [-0.00354759 -0.00082741 -0.00627001 -0.00043818 -0.00477218]
 [-0.01313865  0.00884622  0.00881318  0.01709573  0.00050034]
 [-0.00404677 -0.0054536  -0.01546477  0.00982367 -0.01101068]]
b1 = [[ 0.]
 [ 0.]
 [ 0.]
 [ 0.]]
W2 = [[-0.01185047 -0.0020565   0.01486148  0.00236716]
 [-0.01023785 -0.00712993  0.00625245 -0.00160513]
 [-0.00768836 -0.00230031  0.00745056  0.01976111]]
b2 = [[ 0.]
 [ 0.]
 [ 0.]]


## 4 正向传播模块
### 4.1 Linear Forward
现在你已经实现了初始化参数，接下来将实现正向传播模块。
![正向传播实现思路](http://upload-images.jianshu.io/upload_images/5355764-7ea43cf755c10ca4.png?imageMogr2/auto-orient/strip%7CimageView2/2/w/1240)
练习：实现正向传播过程中的线性计算部分
提示：计算的数学公式为Z[l]=W[l]A[l-1]+b[l]，你会发现np.dot同样很有用，如果你的维度不匹配，打印出W.shape也可以检查。

In [8]:
# GRADED FUNCTION: linear_forward

def linear_forward(A, W, b):
    """
    Implement the linear part of a layer's forward propagation.

    Arguments:
    A -- activations from previous layer (or input data): (size of previous layer, number of examples)
    W -- weights matrix: numpy array of shape (size of current layer, size of previous layer)
    b -- bias vector, numpy array of shape (size of the current layer, 1)

    Returns:
    Z -- the input of the activation function, also called pre-activation parameter 
    cache -- a python dictionary containing "A", "W" and "b" ; stored for computing the backward pass efficiently
    """
    
    ### START CODE HERE ### (≈ 1 line of code)
    Z = np.dot(W,A)+b
    ### END CODE HERE ###
    
    assert(Z.shape == (W.shape[0], A.shape[1]))
    cache = (A, W, b)
    
    return Z, cache

In [9]:
A, W, b = linear_forward_test_case()

Z, linear_cache = linear_forward(A, W, b)
print("Z = " + str(Z))

Z = [[ 3.26295337 -1.23429987]]


注意对上面这一步的理解，这一步只是实现了某一层里面关于线性计算的部分，实现的功能则是输入一个W,b参数以及计算出来的上一层激活值，最后实现计算z，并且在计算z之后把当前这一层的w b以及A都放进cache缓存里面，返回值则为计算出来的Z值以及cache。这里并没有考虑整个神经网络的L层，把一般的能涵盖整个神经网络的线性计算表示出来，因为是针对一层的通用公式，后面再考虑使用循环把整个L层的正向传播都计算出来。

### 4.2 线性计算--激活值计算 正向
![正向传播用到的两个激活函数](http://upload-images.jianshu.io/upload_images/5355764-f45dce5bb8843263.png?imageMogr2/auto-orient/strip%7CimageView2/2/w/1240)
上面提到了sigmoid函数以及relu激活函数，我们已经实现好了在工具包里面，直接使用就行，他们都返回了计算出来的A值，以及新的一个叫做activation_cache的量，里面存放了Z。
根据下面的单隐藏层神经网络梯度下降求法，我们将明白在反向传播过程到底需要哪些值，将它们存放到cache里面。
![单隐藏层神经网络梯度下降导数求法](http://upload-images.jianshu.io/upload_images/5355764-e81ab797f0c4887b.png?imageMogr2/auto-orient/strip%7CimageView2/2/w/1240)

为了方便，你将把计算线性Z值和计算激活值A的函数组合在一起，实现（Linear->activation）的过程，因此，你要实现一个函数，计算Z并且计算A。
![数学表达式](http://upload-images.jianshu.io/upload_images/5355764-6af57f8b214cee7d.png?imageMogr2/auto-orient/strip%7CimageView2/2/w/1240)


In [10]:
# GRADED FUNCTION: linear_activation_forward

def linear_activation_forward(A_prev, W, b, activation):
    """
    Implement the forward propagation for the LINEAR->ACTIVATION layer

    Arguments:
    A_prev -- activations from previous layer (or input data): (size of previous layer, number of examples)
    W -- weights matrix: numpy array of shape (size of current layer, size of previous layer)
    b -- bias vector, numpy array of shape (size of the current layer, 1)
    activation -- the activation to be used in this layer, stored as a text string: "sigmoid" or "relu"

    Returns:
    A -- the output of the activation function, also called the post-activation value 
    cache -- a python dictionary containing "linear_cache" and "activation_cache";
             stored for computing the backward pass efficiently
    """
    
    if activation == "sigmoid":
        # Inputs: "A_prev, W, b". Outputs: "A, activation_cache".
        ### START CODE HERE ### (≈ 2 lines of code)
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = sigmoid(Z)
        ### END CODE HERE ###
    
    elif activation == "relu":
        # Inputs: "A_prev, W, b". Outputs: "A, activation_cache".
        ### START CODE HERE ### (≈ 2 lines of code)
        Z, linear_cache =  linear_forward(A_prev, W, b)
        A, activation_cache = relu(Z)
        ### END CODE HERE ###
    
    assert (A.shape == (W.shape[0], A_prev.shape[1]))
    cache = (linear_cache, activation_cache)

    return A, cache

In [11]:
A_prev, W, b = linear_activation_forward_test_case()

A, linear_activation_cache = linear_activation_forward(A_prev, W, b, activation = "sigmoid")
print("With sigmoid: A = " + str(A))

A, linear_activation_cache = linear_activation_forward(A_prev, W, b, activation = "relu")
print("With ReLU: A = " + str(A))

With sigmoid: A = [[ 0.96890023  0.11013289]]
With ReLU: A = [[ 3.43896131  0.        ]]


### 4.3 L层模型
为了更方便的实现L层神经网络，我们需要一个函数去重复实现linear_activation_forward with RELU L-1次，再跟着实现 linear_activation_forward with SIGMOID一次。
练习：根据上面的模型实现正向传播。
提示：下面的代码中，AL将用来表示A上标[L]，并且A[L]=g(Z[L])=g(W[L]A[L-1]+b[L]),最后的A[L]即AL同样也称为y_hat，即最后预测的值。
步骤：
    1. 使用上面你已经写好的函数
    2. 使用一个Loop循环来实现前面的ReLU应用L-1次
    3. 不要忘记跟踪cache列表里面的内容，即把每次一循环得到的结果存放进一个大的cache列表里面，使用列表追加元素 list.append(c)

In [26]:
# GRADED FUNCTION: L_model_forward

def L_model_forward(X, parameters):
    """
    Implement forward propagation for the [LINEAR->RELU]*(L-1)->LINEAR->SIGMOID computation
    
    Arguments:
    X -- data, numpy array of shape (input size, number of examples)
    parameters -- output of initialize_parameters_deep()
    
    Returns:
    AL -- last post-activation value
    caches -- list of caches containing:
                every cache of linear_relu_forward() (there are L-1 of them, indexed from 0 to L-2)
                the cache of linear_sigmoid_forward() (there is one, indexed L-1)
    """

    caches = []
    A = X
    L = len(parameters) // 2                  # number of layers in the neural network
    
    # Implement [LINEAR -> RELU]*(L-1). Add "cache" to the "caches" list.
    for l in range(1, L):
        A_prev = A 
        ### START CODE HERE ### (≈ 2 lines of code)
        A, cache = linear_activation_forward(A_prev, parameters["W"+str(l)],parameters["b"+str(l)],"relu")
        caches.append(cache)
        ### END CODE HERE ###
    
    # Implement LINEAR -> SIGMOID. Add "cache" to the "caches" list.
    ### START CODE HERE ### (≈ 2 lines of code)
    AL, cache = linear_activation_forward(A, parameters["W"+str(L)],parameters["b"+str(L)], "sigmoid")
    caches.append(cache)
    ### END CODE HERE ###
    
    assert(AL.shape == (1,X.shape[1]))
            
    return AL, caches

In [27]:
X, parameters = L_model_forward_test_case()
AL, caches = L_model_forward(X, parameters)
print("AL = " + str(AL))
print("Length of caches list = " + str(len(caches)))

AL = [[ 0.17007265  0.2524272 ]]
Length of caches list = 2


Great!现在你已经有一个全面的正向传播了，输入为X，输出为A[L]包括了你所有训练样本的预测结果。同样也把所有的中间值都记录到了caches里面，使用A[L],你就可以计算损失值啦。

## 5 代价函数
你需要计算损失，因为你要检测你的模型是否在学习，使得损失在减小。
练习：计算cross-entropy cost J,使用下面这个公式。
![深度神经网络代价函数计算公式](http://upload-images.jianshu.io/upload_images/5355764-16017ad5fc2b260e.png?imageMogr2/auto-orient/strip%7CimageView2/2/w/1240)


In [33]:
# GRADED FUNCTION: compute_cost

def compute_cost(AL, Y):
    """
    Implement the cost function defined by equation (7).

    Arguments:
    AL -- probability vector corresponding to your label predictions, shape (1, number of examples)
    Y -- true "label" vector (for example: containing 0 if non-cat, 1 if cat), shape (1, number of examples)

    Returns:
    cost -- cross-entropy cost
    """
    
    m = Y.shape[1]

    # Compute loss from aL and y.
    ### START CODE HERE ### (≈ 1 lines of code)
    cost = - np.sum(Y*np.log(AL)+(1-Y)*np.log(1-AL))/m
    ### END CODE HERE ###
    
    cost = np.squeeze(cost)      # To make sure your cost's shape is what we expect (e.g. this turns [[17]] into 17).
    assert(cost.shape == ())
    
    return cost

In [34]:
Y, AL = compute_cost_test_case()

print("cost = " + str(compute_cost(AL, Y)))

cost = 0.414931599615


## 6. 反向传播模块
就像正向传播一样，我们需要实现几个辅助函数很好理清思路。记住反向传播视用来计算代价函数对于每个参数的梯度的。
提示：和正向传播过程一样，我们要建立反向传播也分解成三个步骤。
1. 线性backward
2. Linear->activation backward，因为我们的激活函数有两种不同的
3. [LINEAR -> RELU] × (L-1) -> LINEAR -> SIGMOID backward (whole model)

### 6.1 Linear backward
对于第l层，线性部分是计算z值，Z[l]=W[l]A[l-1]+b[l],接下来是使用激活函数计算。假设你已经计算得到了dZ[l]=偏L/偏Z[l],意思即是已经知道了Z的偏导，那么来计算关于参数W[l],b[l],A[l-1]的偏导数，根据链式法则。
![某一层已知dZ[l]计算其他参数导数的公式](http://upload-images.jianshu.io/upload_images/5355764-7bd0bd7682a2a8b6.png?imageMogr2/auto-orient/strip%7CimageView2/2/w/1240)
练习：使用上面的三个公式来计算线性部分的参数的偏导数。


In [42]:
# GRADED FUNCTION: linear_backward

def linear_backward(dZ, cache):
    """
    Implement the linear portion of backward propagation for a single layer (layer l)

    Arguments:
    dZ -- Gradient of the cost with respect to the linear output (of current layer l)
    cache -- tuple of values (A_prev, W, b) coming from the forward propagation in the current layer

    Returns:
    dA_prev -- Gradient of the cost with respect to the activation (of the previous layer l-1), same shape as A_prev
    dW -- Gradient of the cost with respect to W (current layer l), same shape as W
    db -- Gradient of the cost with respect to b (current layer l), same shape as b
    """
    A_prev, W, b = cache
    m = A_prev.shape[1]

    ### START CODE HERE ### (≈ 3 lines of code)
    dW = np.dot(dZ,A_prev.T)/m
    db = np.sum(dZ, axis=1, keepdims=True)/m
    dA_prev = np.dot(W.T,dZ)
    ### END CODE HERE ###
    
    assert (dA_prev.shape == A_prev.shape)
    assert (dW.shape == W.shape)
    assert (db.shape == b.shape)
    
    return dA_prev, dW, db

In [43]:
# Set up some test inputs
dZ, linear_cache = linear_backward_test_case()

dA_prev, dW, db = linear_backward(dZ, linear_cache)
print ("dA_prev = "+ str(dA_prev))
print ("dW = " + str(dW))
print ("db = " + str(db))

dA_prev = [[ 0.51822968 -0.19517421]
 [-0.40506361  0.15255393]
 [ 2.37496825 -0.89445391]]
dW = [[-0.10076895  1.40685096  1.64992505]]
db = [[ 0.50629448]]


### 6.2 Linear-activation backward
接下来，你将创建一个函数把两个辅助函数整合在一起，则是在线性部分计算导数的以及激活函数也计算导数的，最后得到linear_activation_backward。

为了帮助你实现linear_activation_backward，在工具包里面直接提供了两种激活函数的导数计算的函数。

![激活函数求导辅助函数内容](http://upload-images.jianshu.io/upload_images/5355764-a30d4730b7ab92a3.png?imageMogr2/auto-orient/strip%7CimageView2/2/w/1240)
我们可以发现，要想计算当前层dZ[l],那么就得通过A=g(Z)来求，根据求导链式法则，需要的参数有dA[l]以及g(Z[l])导。
在工具包直接提供的辅助函数里面，传入的参数为dA，以及activation_cache，因为这个缓存里面存放了当前层计算出来的Z[l]值。

练习：实现反向传播Linear->activation过程


In [44]:
# GRADED FUNCTION: linear_activation_backward

def linear_activation_backward(dA, cache, activation):
    """
    Implement the backward propagation for the LINEAR->ACTIVATION layer.
    
    Arguments:
    dA -- post-activation gradient for current layer l 
    cache -- tuple of values (linear_cache, activation_cache) we store for computing backward propagation efficiently
    activation -- the activation to be used in this layer, stored as a text string: "sigmoid" or "relu"
    
    Returns:
    dA_prev -- Gradient of the cost with respect to the activation (of the previous layer l-1), same shape as A_prev
    dW -- Gradient of the cost with respect to W (current layer l), same shape as W
    db -- Gradient of the cost with respect to b (current layer l), same shape as b
    """
    linear_cache, activation_cache = cache
    
    if activation == "relu":
        ### START CODE HERE ### (≈ 2 lines of code)
        dZ = relu_backward(dA,activation_cache)
        dA_prev, dW, db = linear_backward(dZ,linear_cache)
        ### END CODE HERE ###
        
    elif activation == "sigmoid":
        ### START CODE HERE ### (≈ 2 lines of code)
        dZ = sigmoid_backward(dA,activation_cache)
        dA_prev, dW, db = linear_backward(dZ,linear_cache)
        ### END CODE HERE ###
    
    return dA_prev, dW, db

In [45]:
AL, linear_activation_cache = linear_activation_backward_test_case()

dA_prev, dW, db = linear_activation_backward(AL, linear_activation_cache, activation = "sigmoid")
print ("sigmoid:")
print ("dA_prev = "+ str(dA_prev))
print ("dW = " + str(dW))
print ("db = " + str(db) + "\n")

dA_prev, dW, db = linear_activation_backward(AL, linear_activation_cache, activation = "relu")
print ("relu:")
print ("dA_prev = "+ str(dA_prev))
print ("dW = " + str(dW))
print ("db = " + str(db))

sigmoid:
dA_prev = [[ 0.11017994  0.01105339]
 [ 0.09466817  0.00949723]
 [-0.05743092 -0.00576154]]
dW = [[ 0.10266786  0.09778551 -0.01968084]]
db = [[-0.05729622]]

relu:
dA_prev = [[ 0.44090989 -0.        ]
 [ 0.37883606 -0.        ]
 [-0.2298228   0.        ]]
dW = [[ 0.44513824  0.37371418 -0.10478989]]
db = [[-0.20837892]]


### 6.3 L层反向传播
现在你将要实现整个L层神经网络的反向传播函数。回想一下，当你实现 L_model_forward时如何完成的，在每一次迭代，你会存下一个cache，里面包含了（X，W，b，以及Z值），在反向传播魔铠中，我们将使用到这些变量去计算梯度。因此，在L层反向传播模块L_model_backward函数中，你将对多有隐藏层进行迭代，从最后一层L层开始。每一迭代过程中，你将使用cache中第l层存好的值来计算。

初始化反向传播：在正向传播过程中，我们知道初始值A0其实就是X，那么同样的，反向过程，我们要知道初始值是什么，那么就应该是最后一层的dA[L]偏导，如何计算呢，使用下面的公式：
![深度神经网络反向传播初始值dA[L]计算公式](http://upload-images.jianshu.io/upload_images/5355764-2fc48533f290d83a.png?imageMogr2/auto-orient/strip%7CimageView2/2/w/1240)
推导过程，如何得到这个结果的呢，首先考虑一个样本，那么对于J代价函数对A求导，计算结果如下：
![代价函数对最后一层A求导](http://upload-images.jianshu.io/upload_images/5355764-0bd98d2ac89d538e.png?imageMogr2/auto-orient/strip%7CimageView2/2/w/1240)
所以要使用向量化方法同时对m个样本进行计算，其计算方法就是把每个样本的导数值计算起来，横向堆叠，所以向量化操作，使用np.divide()方法即可。

接下来的步骤，就是使用dA[L]放进LINEAR->SIGMOID backward函数，再使用for循环对前面的L-1层使用Linear->relu的梯度计算。这些过程中用到的参数都是从正向传播存好的cache中取得的。同样的，为了后面迭代很多次对参数进行更新，反向传播计算出来的梯度值，我们也需要存储起来，因此将dA，dW，db存放到grads字典里面。参数的不同层标记我们都是大写字母后面加个数字直接存放的，所以例子如下：**grads["dW"+str(l)]=dW[l] **
不像正向传播，对于超参数等的存放都一层作为一个单位，形成很多个cache，再放到caches列表里面。

In [46]:
# GRADED FUNCTION: L_model_backward

def L_model_backward(AL, Y, caches):
    """
    Implement the backward propagation for the [LINEAR->RELU] * (L-1) -> LINEAR -> SIGMOID group
    
    Arguments:
    AL -- probability vector, output of the forward propagation (L_model_forward())
    Y -- true "label" vector (containing 0 if non-cat, 1 if cat)
    caches -- list of caches containing:
                every cache of linear_activation_forward() with "relu" (it's caches[l], for l in range(L-1) i.e l = 0...L-2)
                the cache of linear_activation_forward() with "sigmoid" (it's caches[L-1])
    
    Returns:
    grads -- A dictionary with the gradients
             grads["dA" + str(l)] = ... 
             grads["dW" + str(l)] = ...
             grads["db" + str(l)] = ... 
    """
    grads = {}
    L = len(caches) # the number of layers
    m = AL.shape[1]
    Y = Y.reshape(AL.shape) # after this line, Y is the same shape as AL
    
    # Initializing the backpropagation
    ### START CODE HERE ### (1 line of code)
    dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL))
    ### END CODE HERE ###
    
    # Lth layer (SIGMOID -> LINEAR) gradients. Inputs: "AL, Y, caches". Outputs: "grads["dAL"], grads["dWL"], grads["dbL"]
    ### START CODE HERE ### (approx. 2 lines)
    current_cache = caches[L-1]
    grads["dA" + str(L)], grads["dW" + str(L)], grads["db" + str(L)] = linear_activation_backward(dAL, current_cache, "sigmoid")
    ### END CODE HERE ###
    
    for l in reversed(range(L-1)):
        # lth layer: (RELU -> LINEAR) gradients.
        # Inputs: "grads["dA" + str(l + 2)], caches". Outputs: "grads["dA" + str(l + 1)] , grads["dW" + str(l + 1)] , grads["db" + str(l + 1)] 
        ### START CODE HERE ### (approx. 5 lines)
        current_cache = caches[l]
        dA_prev_temp, dW_temp, db_temp =  linear_activation_backward(grads["dA"+str(l+2)], current_cache, "relu")
        grads["dA" + str(l + 1)] = dA_prev_temp
        grads["dW" + str(l + 1)] = dW_temp
        grads["db" + str(l + 1)] = db_temp
        ### END CODE HERE ###

    return grads

In [47]:
AL, Y_assess, caches = L_model_backward_test_case()
grads = L_model_backward(AL, Y_assess, caches)
print ("dW1 = "+ str(grads["dW1"]))
print ("db1 = "+ str(grads["db1"]))
print ("dA1 = "+ str(grads["dA1"]))

dW1 = [[ 0.41010002  0.07807203  0.13798444  0.10502167]
 [ 0.          0.          0.          0.        ]
 [ 0.05283652  0.01005865  0.01777766  0.0135308 ]]
db1 = [[-0.22007063]
 [ 0.        ]
 [-0.02835349]]
dA1 = [[ 0.          0.52257901]
 [ 0.         -0.3269206 ]
 [ 0.         -0.32070404]
 [ 0.         -0.74079187]]


### 6.4 更新参数
在这一节中你将要使用计算出来的梯度来更新参数
1. W[l]=W[l]−α dW[l] 
2. b[l]=b[l]−α db[l] 

并且为了循环迭代更新，更新好的参数仍然要存放在parameters字典里面
练习：实现update_parameters()来使用梯度下降法更新你的参数


In [48]:
# GRADED FUNCTION: update_parameters

def update_parameters(parameters, grads, learning_rate):
    """
    Update parameters using gradient descent
    
    Arguments:
    parameters -- python dictionary containing your parameters 
    grads -- python dictionary containing your gradients, output of L_model_backward
    
    Returns:
    parameters -- python dictionary containing your updated parameters 
                  parameters["W" + str(l)] = ... 
                  parameters["b" + str(l)] = ...
    """
    
    L = len(parameters) // 2 # number of layers in the neural network

    # Update rule for each parameter. Use a for loop.
    ### START CODE HERE ### (≈ 3 lines of code)
    for l in range(L):
        parameters["W" + str(l+1)] =  parameters["W" + str(l+1)]-learning_rate*grads["dW"+str(l+1)]
        parameters["b" + str(l+1)] =parameters["b" + str(l+1)]-learning_rate*grads["db"+str(l+1)]
    ### END CODE HERE ###
        
    return parameters

In [49]:
parameters, grads = update_parameters_test_case()
parameters = update_parameters(parameters, grads, 0.1)

print ("W1 = "+ str(parameters["W1"]))
print ("b1 = "+ str(parameters["b1"]))
print ("W2 = "+ str(parameters["W2"]))
print ("b2 = "+ str(parameters["b2"]))

W1 = [[-0.59562069 -0.09991781 -2.14584584  1.82662008]
 [-1.76569676 -0.80627147  0.51115557 -1.18258802]
 [-1.0535704  -0.86128581  0.68284052  2.20374577]]
b1 = [[-0.04659241]
 [-1.28888275]
 [ 0.53405496]]
W2 = [[-0.55569196  0.0354055   1.32964895]]
b2 = [[-0.84610769]]


## 7 总结
已经实现了建立一个深度神经网络需要的所有函数，我们知道这是一个很长的任务，坚持完成下去，将会变得更好。
在下一节任务week4_03中，我们将会把上面的融合在一起建立两种模型：
1. 双层神经网络模型
2. L层神经网络模型
事实上你会使用这些模型来对猫图和非猫图进行分类。

暂时还没有实际应用到数据集上，但是已经完成了实现L层神经网络的整个过程需要的函数，对于这节任务，收获还是很多，其中最主要的就是如何将理论转化成代码，如何变成数字符号，让电脑去运作。而且理论部分，我们提到了正向和反向传播过程需要的参数，需要存放进cache存储，已经两个过程的初始值，在完成这节任务前，我都觉得很难用代码实现，写成代码的逻辑会不清楚，总结的说，以下几点很重要：
1. 分解过程，将正向传播的L层实现，分三步完成，首先第一步计算z，第二步，将计算z和计算A结合（在这里也就实现了对两种不同激活函数的不同运算，第三步，对L层的前L-1层使用循环计算，最后一层加上sigmoid的运算。这样分解步骤的好处在于，层层递进，为下一步提供参数，并且由于我们知道后面反向传播会用到前面计算出来的值，在分解的过程我们就对参数值进行了存储，在字典或者元祖里面，很巧妙。当然，对于反向传播过程的分解，也同理相似，反向传播过程则是前面的cache作为参数，输出就相对简单一些，只需要把梯度值存起来放到grads字典里。
2. 关于变量类型选择，对于参数W，b,因为要使用的比较多，所以采用字母后面加数字来表示每一层，全都存放在一个parameters字典里，而对于其余的超参数，例如，A，W，b作为一组，计算z放在一个linear_cache里面，再加上激活函数，计算过后，把Z也和前面三个参数一起存放到另一个linear_activation_cache字典里。因为后面L层实现，对每一层都要起作用，全放在一个字典修改也比较麻烦，因此将每一层的所有cache作为一个整理，每迭代一次一层，就把这一层的cache都放进一个caches列表里面，在for循环每一层时，再根据下角标依次取出来。又比如说，对于层数，我们要进行循环，因此使用列表，将每一个隐藏层的节点数存在列表里，对列表长度进行循环每一层，取出每层节点数也简单方便。

总之，关于深度神经网络的实现就是一个复杂的过程，通过逐个函数实现，明确每个函数功能，最后再整合，可能把问题简单化，让每个函数实现的功能尽量单一，最后将小函数合成大函数，也就完成我们的模型的建立。